# OPTaaS Quick Start

### <span style="color:red">Note:</span> To run this notebook, you need an API Key. You can get one <a href="mailto:charles.brecque@mindfoundry.ai">here</a>.

More tutorials are [available here](./)

## Connect to OPTaaS using your API Key

In [1]:
from mindfoundry.optaas.client.client import OPTaaSClient

client = OPTaaSClient('https://optaas.mindfoundry.ai', '<Your OPTaaS API key>')

## Define your parameters

In [2]:
from mindfoundry.optaas.client.parameter import IntParameter, FloatParameter, CategoricalParameter, BoolParameter, \
    ChoiceParameter, GroupParameter

bool_param = BoolParameter('my_bool')
cat_param = CategoricalParameter('my_cat', values=['a', 'b', 'c'], default='c')

int_param = IntParameter('my_int', minimum=0, maximum=20)
optional_int_param = IntParameter('my_optional_int', minimum=-10, maximum=10, optional=True)

parameters = [
    bool_param,
    cat_param,
    ChoiceParameter('ints_or_floats', choices=[
        GroupParameter('ints', items=[int_param, optional_int_param]),
        GroupParameter('floats', items=[
            FloatParameter('float1', minimum=0, maximum=1),
            FloatParameter('float2', minimum=0.5, maximum=4.5)
        ])
    ]),
]

## Define your scoring function

The argument names in your scoring function must match the parameter names you defined above.

Your function can return just a score, or a tuple of (score, variance).

In [3]:
def scoring_function(my_bool, my_cat, ints_or_floats):
    score = 5 if my_bool is True else -5
    score += 1 if my_cat == 'a' else 3
    if 'ints' in ints_or_floats:
        score += sum(ints_or_floats['ints'].values())
    else:
        score *= sum(ints_or_floats['floats'].values())
    return score

## Create your Task

You can use Goal.max or Goal.min as appropriate. You can also specify the best possible score (if known).

In [4]:
from mindfoundry.optaas.client.client import Goal

task = client.create_task(
    title='Quick Start Example Task',
    parameters=parameters,
    goal=Goal.max,
    target_score=44.0
)

## Run your Task

We will run for a maximum of 50 iterations, but we will stop as soon as we reach our score threshold of 32.

The score threshold is optional - you can omit it and simply run as many iterations as you need.

In [5]:
best_result = task.run(scoring_function, max_iterations=50, score_threshold=32)
print("Best Result:", best_result)

Running task "Quick Start Example Task" for 50 iterations
(or until score is 32 or better)

Iteration: 0    Score: 8
Configuration: {'my_bool': False, 'my_cat': 'c', 'ints_or_floats': {'ints': {'my_int': 10, 'my_optional_int': 0}}}

Iteration: 1    Score: 22
Configuration: {'my_bool': True, 'my_cat': 'c', 'ints_or_floats': {'ints': {'my_int': 14}}}

Iteration: 2    Score: 7
Configuration: {'my_bool': False, 'my_cat': 'a', 'ints_or_floats': {'ints': {'my_int': 14, 'my_optional_int': -3}}}

Iteration: 3    Score: 7
Configuration: {'my_bool': False, 'my_cat': 'c', 'ints_or_floats': {'ints': {'my_int': 9}}}

Iteration: 4    Score: 28
Configuration: {'my_bool': True, 'my_cat': 'c', 'ints_or_floats': {'ints': {'my_int': 20}}}

Iteration: 5    Score: 17
Configuration: {'my_bool': True, 'my_cat': 'c', 'ints_or_floats': {'ints': {'my_int': 1, 'my_optional_int': 8}}}

Iteration: 6    Score: 13
Configuration: {'my_bool': True, 'my_cat': 'b', 'ints_or_floats': {'ints': {'my_int': 5}}}

Iteration: 